# 건설자재 품질검사 등록정보

In [1]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "건설자재 품질검사 등록정보"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv'

In [2]:
APIKEYLEN = len(APIKEY)

In [3]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [4]:
JSONKEY = "items"
DUMMY = 0

In [5]:
# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###
PAGEYN

1

In [6]:
resultDfMerged = pd.DataFrame()

In [7]:
def saveLastSearchYr(paramStartYr):
    cf.saveparam(paramStartYr, SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHYR")
    print("Last Search Yr {} is saved.".format(paramStartYr))
def loadLastSearchYr():
    ret = cf.loadparam(SITENAME, DATANAME, SERVICENAME+"_LASTSEARCHYR")
    print(ret+" is loaded")
    return ret

In [8]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endYr

'2022'

In [9]:
# 기준년도 산정
# 2001년 부터 데이터 존재
startYr = "2001"
mode = 2
try:
    startYr = loadLastSearchYr()
    mode = 1
except Exception as e:
    print(e)
# startYr = "2001" # <- 최초 데이터 수집 이후 주석처리해야 함
dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
dateRange

[Errno 2] No such file or directory: '../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList_LASTSEARCHYR.pickle'


[2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

## 건설자재 품질검사 등록정보 전용 스크랩 함수

In [ ]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            time.sleep(1.2)
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [10]:
resultDfMerged = pd.DataFrame()

# 건설자재 품질검사 등록정보 목록 수
# g_totalCount = 0

In [13]:
numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
resultDf = pd.DataFrame()
APICALL = 0 #추가
pageNo = 1
pageList = [] #추가
print("len of dateRange : ",len(dateRange))
for i in range(0,len(dateRange)):

    print("i=",i)
    year = dateRange[i]
#     apiNo = i%APIKEYLEN
#     g_totalCount = 0
    
#     while True:
    BASEPARAM={"serviceKey":APIKEY[0],"year": year, "sortField": "tstBgDt","sortOrder": "ASC", "pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
    print("BASEPARAM ",BASEPARAM)
    scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, APIKEY,APICALL, JSONKEY, DUMMY)
    resultDf = scrapyResult[0]
    pageList.append( scrapyResult[1] )
    APICALL = scrapyResult[2]
    if resultDf.empty: # 정상 데이터가 없는 경우
        print("No Data")
#         break
    else:
        resultDfMerged = resultDfMerged.append(resultDf)
#             g_totalCount -= numOfRows
#             print("g_totalCount : ", g_totalCount)
#             if(g_totalCount <= 0):
#                 break
#             pageNo += 1

len of dateRange :  22
i= 0
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'year': 2001, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
totalCount 3
2 page scraping start apicall iter: 2 / used 0974320D-C689-4543-8464-10DE1E5505A9
2 page is empty
dataframe건설자재 품질검사 등록정보, param:{'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'year': 2001, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 2, 'numOfRows': 1000, 'type': 'json'} rows: 18 completed
i= 1
BASEPARAM  {'serviceKey': '0974320D-C689-4543-8464-10DE1E5505A9', 'year': 2002, 'sortField': 'tstBgDt', 'sortOrder': 'ASC', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start apicall iter: 3 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
1 page is empty
dataframe건설자재 품질검사 등록정보, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'year': 2002, 'sortField': 'tstBgDt

6 page scraping start apicall iter: 52 / used 0974320D-C689-4543-8464-10DE1E5505A9
7 page scraping start apicall iter: 53 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
8 page scraping start apicall iter: 54 / used 0974320D-C689-4543-8464-10DE1E5505A9
9 page scraping start apicall iter: 55 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
10 page scraping start apicall iter: 56 / used 0974320D-C689-4543-8464-10DE1E5505A9
11 page scraping start apicall iter: 57 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
12 page scraping start apicall iter: 58 / used 0974320D-C689-4543-8464-10DE1E5505A9
13 page scraping start apicall iter: 59 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
14 page scraping start apicall iter: 60 / used 0974320D-C689-4543-8464-10DE1E5505A9
15 page scraping start apicall iter: 61 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
16 page scraping start apicall iter: 62 / used 0974320D-C689-4543-8464-10DE1E5505A9
17 page scraping start apicall iter: 63 / used 49099D1D-E565-48E1-8A6F-D5F47135E

103 page scraping start apicall iter: 149 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
104 page scraping start apicall iter: 150 / used 0974320D-C689-4543-8464-10DE1E5505A9
105 page scraping start apicall iter: 151 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
106 page scraping start apicall iter: 152 / used 0974320D-C689-4543-8464-10DE1E5505A9
107 page scraping start apicall iter: 153 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
108 page scraping start apicall iter: 154 / used 0974320D-C689-4543-8464-10DE1E5505A9
109 page scraping start apicall iter: 155 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
110 page scraping start apicall iter: 156 / used 0974320D-C689-4543-8464-10DE1E5505A9
111 page scraping start apicall iter: 157 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
112 page scraping start apicall iter: 158 / used 0974320D-C689-4543-8464-10DE1E5505A9
113 page scraping start apicall iter: 159 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
114 page scraping start apicall iter: 160 / used 09743

67 page scraping start apicall iter: 241 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
68 page scraping start apicall iter: 242 / used 0974320D-C689-4543-8464-10DE1E5505A9
69 page scraping start apicall iter: 243 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
70 page scraping start apicall iter: 244 / used 0974320D-C689-4543-8464-10DE1E5505A9
71 page scraping start apicall iter: 245 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
72 page scraping start apicall iter: 246 / used 0974320D-C689-4543-8464-10DE1E5505A9
73 page scraping start apicall iter: 247 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
74 page scraping start apicall iter: 248 / used 0974320D-C689-4543-8464-10DE1E5505A9
75 page scraping start apicall iter: 249 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
76 page scraping start apicall iter: 250 / used 0974320D-C689-4543-8464-10DE1E5505A9
77 page scraping start apicall iter: 251 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
78 page scraping start apicall iter: 252 / used 0974320D-C689-454

35 page scraping start apicall iter: 333 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
36 page scraping start apicall iter: 334 / used 0974320D-C689-4543-8464-10DE1E5505A9
37 page scraping start apicall iter: 335 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
38 page scraping start apicall iter: 336 / used 0974320D-C689-4543-8464-10DE1E5505A9
39 page scraping start apicall iter: 337 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
40 page scraping start apicall iter: 338 / used 0974320D-C689-4543-8464-10DE1E5505A9
41 page scraping start apicall iter: 339 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
42 page scraping start apicall iter: 340 / used 0974320D-C689-4543-8464-10DE1E5505A9
43 page scraping start apicall iter: 341 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
44 page scraping start apicall iter: 342 / used 0974320D-C689-4543-8464-10DE1E5505A9
45 page scraping start apicall iter: 343 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
46 page scraping start apicall iter: 344 / used 0974320D-C689-454

...

98 page scraping start apicall iter: 896 / used 0974320D-C689-4543-8464-10DE1E5505A9
99 page scraping start apicall iter: 897 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
100 page scraping start apicall iter: 898 / used 0974320D-C689-4543-8464-10DE1E5505A9
101 page scraping start apicall iter: 899 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
102 page scraping start apicall iter: 900 / used 0974320D-C689-4543-8464-10DE1E5505A9
103 page scraping start apicall iter: 901 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
104 page scraping start apicall iter: 902 / used 0974320D-C689-4543-8464-10DE1E5505A9
105 page scraping start apicall iter: 903 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
106 page scraping start apicall iter: 904 / used 0974320D-C689-4543-8464-10DE1E5505A9
107 page scraping start apicall iter: 905 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
108 page scraping start apicall iter: 906 / used 0974320D-C689-4543-8464-10DE1E5505A9
109 page scraping start apicall iter: 907 / used 49099D1

56 page scraping start apicall iter: 988 / used 0974320D-C689-4543-8464-10DE1E5505A9
57 page scraping start apicall iter: 989 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
58 page scraping start apicall iter: 990 / used 0974320D-C689-4543-8464-10DE1E5505A9
59 page scraping start apicall iter: 991 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
60 page scraping start apicall iter: 992 / used 0974320D-C689-4543-8464-10DE1E5505A9
61 page scraping start apicall iter: 993 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
62 page scraping start apicall iter: 994 / used 0974320D-C689-4543-8464-10DE1E5505A9
63 page scraping start apicall iter: 995 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
64 page scraping start apicall iter: 996 / used 0974320D-C689-4543-8464-10DE1E5505A9
65 page scraping start apicall iter: 997 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
66 page scraping start apicall iter: 998 / used 0974320D-C689-4543-8464-10DE1E5505A9
67 page scraping start apicall iter: 999 / used 49099D1D-E565-48E

20 page scraping start apicall iter: 1079 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
21 page scraping start apicall iter: 1080 / used 0974320D-C689-4543-8464-10DE1E5505A9
22 page scraping start apicall iter: 1081 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
23 page scraping start apicall iter: 1082 / used 0974320D-C689-4543-8464-10DE1E5505A9
24 page scraping start apicall iter: 1083 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
25 page scraping start apicall iter: 1084 / used 0974320D-C689-4543-8464-10DE1E5505A9
26 page scraping start apicall iter: 1085 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
27 page scraping start apicall iter: 1086 / used 0974320D-C689-4543-8464-10DE1E5505A9
28 page scraping start apicall iter: 1087 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
29 page scraping start apicall iter: 1088 / used 0974320D-C689-4543-8464-10DE1E5505A9
30 page scraping start apicall iter: 1089 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
31 page scraping start apicall iter: 1090 / used 09743

116 page scraping start apicall iter: 1175 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
117 page scraping start apicall iter: 1176 / used 0974320D-C689-4543-8464-10DE1E5505A9
118 page scraping start apicall iter: 1177 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
119 page scraping start apicall iter: 1178 / used 0974320D-C689-4543-8464-10DE1E5505A9
120 page scraping start apicall iter: 1179 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
121 page scraping start apicall iter: 1180 / used 0974320D-C689-4543-8464-10DE1E5505A9
122 page scraping start apicall iter: 1181 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
123 page scraping start apicall iter: 1182 / used 0974320D-C689-4543-8464-10DE1E5505A9
124 page scraping start apicall iter: 1183 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
125 page scraping start apicall iter: 1184 / used 0974320D-C689-4543-8464-10DE1E5505A9
126 page scraping start apicall iter: 1185 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
127 page scraping start apicall iter: 1186 

61 page scraping start apicall iter: 1256 / used 0974320D-C689-4543-8464-10DE1E5505A9
62 page scraping start apicall iter: 1257 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
63 page scraping start apicall iter: 1258 / used 0974320D-C689-4543-8464-10DE1E5505A9
64 page scraping start apicall iter: 1259 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
65 page scraping start apicall iter: 1260 / used 0974320D-C689-4543-8464-10DE1E5505A9
66 page scraping start apicall iter: 1261 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
67 page scraping start apicall iter: 1262 / used 0974320D-C689-4543-8464-10DE1E5505A9
68 page scraping start apicall iter: 1263 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
69 page scraping start apicall iter: 1264 / used 0974320D-C689-4543-8464-10DE1E5505A9
70 page scraping start apicall iter: 1265 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
71 page scraping start apicall iter: 1266 / used 0974320D-C689-4543-8464-10DE1E5505A9
72 page scraping start apicall iter: 1267 / used 49099

In [14]:
resultDfMerged

,rn,mtlnm,perno,ornm,cwknm,ordrnm,cstrnm,stwrdt,ccwxpcdt,tstbgdt,tsteddt,rpcdisdt,smorsecdnm,smornm,prdcntnnm,smorpcknplacnm,otcuspucts,altsitnm
0,1,한솔검사엔지니어링(주),제2008-12호,서울지방국토관리청,벡스코 시설확충사업 신축공사,None,현대건설(주),None,None,2001-01-17,2001-01-17,2001-01-17,None,None,None,None,None,UT
1,2,한솔검사엔지니어링(주),제2008-12호,서울지방국토관리청,벡스코 시설확충사업 신축공사,None,현대건설(주),None,None,2001-01-17,2001-01-17,2001-01-17,None,None,None,None,None,MT
2,3,(주)다우엔지니어링,제2010-23호,서울지방국토관리청,송우~무봉간 도로 확포장 건설공사,포천시청,일신종합건설㈜,None,None,2001-10-29,2001-11-06,2001-11-06,None,폐콘크리트,None,현장 내(STA. 1+780~1+820),품질시험,None
0,1,한솔검사엔지니어링(주),제2008-12호,서울지방국토관리청,벡스코 시설확충사업 신축공사,None,현대건설(주),None,None,2001-01-17,2001-01-17,2001-01-17,None,None,None,None,None,UT
1,2,한솔검사엔지니어링(주),제2008-12호,서울지방국토관리청,벡스코 시설확충사업 신축공사,None,현대건설(주),None,None,2001-01-17,2001-01-17,2001-01-17,None,None,None,None,None,MT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11996,(주)한국건설품질기술원,경기1-3-37호,경기도청,동북선 도시철도 민간투자사업 건설공사(4공구),동북선도시철도 주식회사,코오롱글로벌(주),2021-07-15,2026-07-14,2021-12-28,2022-01-04,2022-01-04,기타 건설자재ㆍ부재,무늬 H형 복공판(1990LX985WX199H),대한민국,동북선 도시철도 입출고선,품질관리용,5mm변위시하중
996,11997,한국건설재료연구원 주식회사,광주-3-10호,광주광역시청,망호항 어촌뉴딜300사업 생활soc분야 토목공사,강진군 한국농어촌공사 전남지역본부 강진지사,(주)영진토건,2021-06-28,2022-12-20,2021-12-28,2022-01-06,2022-01-07,기타 건설자재ㆍ부재,보강점토,대한민국,망호항 현장내,품질시험,압축강도
997,11998,주식회사 한국도로시험연구원,경기1-3-69호,경기도청,고천지구 하천재해예방사업,경주시청,㈜삼건사,2021-02-01,2021-12-31,2021-12-28,2022-01-04,2022-01-04,기타 건설자재ㆍ부재,보조기층,대한민국,"현장내(제3화천교 진입도로 시점, 제3화천교 진입도로 종점)",품질시험,도로평판
998,11999,한국건설재료연구원 주식회사,광주-3-10호,광주광역시청,완도지구 지표수보강개발사업,한국농어촌공사 해남완도지사,운정건설(주),2020-08-10,2021-12-20,2021-12-28,2022-01-04,2022-01-05,기타 건설자재ㆍ부재,콘크리트,대한민국,전남 완도군 군외면 대문리 652-3일원,품질시험,압축강도


In [15]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME,mode)

건설자재 품질검사 등록정보 save compled


In [16]:
resultDfMerged.shape

(1263427, 18)

### 다음 회차 수행을 위해 현재 검색종료 Year를 저장한다. 
### 다음 회차 검색은 현재 저장한 검색종료 Year가 시작점이 된다. 

In [17]:
# 1년 차감하여 저장, 당해 년은 계속 데이터가 쌓이기 때문
saveLastSearchYr(str(int(endYr)-1))

Last Search Yr 2021 is saved.
